In [1]:
import random
import numpy as np
from collections import deque
import numpy as np
import pandas as pd
from collections import defaultdict


In [2]:

def epsilon_greedy_search(Epsilon, qtable, state):
    if (random.random() < Epsilon):
        # 0 is 'apply matrix A', 1 is 'apply matrix B'
        # 2 is 'apply matrix C', 3 is 'apply matrix D'
        return random.choice([0, 1, 2, 3])
    else:
        # get the best move for the current state
        return best_move_for_a_state(Q_table=qtable, state=state)
    
# I would like to return the best move for a given state
def best_move_for_a_state(Q_table, state):
    vals = Q_table[(state[0][1], state[0][2], state[1][2])]

    # if we haven't visited this state before, return a random choice of 0, 1, 2, or 3
    if vals==[0, 0, 0, 0]:
        return random.choice([0, 1, 2, 3])
    
    # if we have visited this state before, return the current best choice
    return np.argmax(vals)

# over a given state, return the maximum value of the table for that state
def max_a_prime(Q_table, state):
    return max(Q_table[(state[0][1], state[0][2], state[1][2])])

In [3]:
# B is the inverse of A
A = np.array([[1, 1, 0], [0, 1, 0], [0, 0, 1]])
B = np.array([[1, -1, 0], [0, 1, 0], [0, 0, 1]])

# C is the inverse of D
C = np.array([[1, 0, 0], [0, 1, 1], [0, 0, 1]])
D = np.array([[1, 0, 0], [0, 1, -1], [0, 0, 1]])

# together, A, B, C, and D generate the heisenberg group

In [4]:
def getReward(matrix):
    if (matrix==A).all() or (matrix==B).all() or (matrix==C).all() or (matrix==D).all():
        return 20
    else:
        return -1 + 1/(2 + abs(matrix[0][1]) + abs(matrix[0][2]) + abs(matrix[1][2]))

In [5]:
# adapted from ChatGPT
class CustomDefaultDict(dict):
    def __init__(self, default_factory, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.default_factory = default_factory

    def __missing__(self, key):
        # Compute the default value based on the missing key
        default_value = self.default_factory(key)
        self[key] = default_value  # Cache the default value for future lookups
        return default_value

# Example usage:
def default_value_for_key(key):
    # Define a function that computes the default value based on the key
    val1 = key[0]
    val2 = key[1]
    val3 = key[2]
    cur_matrix = np.array([[1, val1, val2], [0, 1, val3], [0, 0, 1]])
    output1 = getReward(cur_matrix @ A)
    output2 = getReward(cur_matrix @ B)
    output3 = getReward(cur_matrix @ C)
    output4 = getReward(cur_matrix @ D)
    return [output1, output2, output3, output4]

custom_dict = CustomDefaultDict(default_value_for_key)

cur_mat = A@A@B

custom_dict[(cur_mat[0][1], cur_mat[0][2], cur_mat[1][2])]

[-0.75, -0.5, -0.8, -0.8]

In [6]:
df = pd.read_csv("heisenberg_data.csv")

In [7]:
def get_next_step(oldObs, action):
    # action is always either 0, 1, 2, or 3
    next_state = []
    if action==0:
        next_state = oldObs @ A
    elif action==1:
        next_state = oldObs @ B
    elif action==2:
        next_state = oldObs @ C
    else:
        next_state = oldObs @ D
    curReward = getReward(next_state)
    done = curReward==20
    return (next_state, curReward, done)
    

In [8]:
# adapted from CS 540 Spring 2023 HW 10
EPISODES = 30000
LEARNING_RATE = .1
DISCOUNT_FACTOR = .99
EPSILON = 1
EPSILON_DECAY = .999

random.seed(1)

# starts with an estimate of zero reward for each state.
# adapted from ChatGPT
# the outer dictionary has keys of the string version of the given array, and 
# values of a dictionary for each of the actions that we could take at that state
# Q_table = defaultdict(lambda: [0, 0, 0, 0])
Q_table = CustomDefaultDict(default_value_for_key)

episode_reward_record = deque(maxlen=100)

for i in range(EPISODES):
    episode_reward = 0
    done = False
    # choose a random starting row
    # adapted from https://stackoverflow.com/questions/15923826/random-row-selection-in-pandas-dataframe
    cur_row = df.sample(1)
    obs = np.array([
        [1, int(cur_row['val1']), int(cur_row['val2'])], 
        [0, 1, int(cur_row['val3'])], 
        [0, 0, 1]
        ])

    index = 1

    while (not done):
        # perform an epsilon greedy action 
        # Q(s, a) = (1-LEARNING_RATE)Q(s, a) + (LEARNING_RATE)(r + DISCOUNT_FACTOR(max a'(Q(s', a'))))
        action = epsilon_greedy_search(Epsilon=EPSILON, qtable=Q_table, state=obs)

        oldObs = obs
        obs,reward,done = get_next_step(oldObs, action)
        Q_table[(oldObs[0][1], oldObs[0][2], oldObs[1][2])][action] = (1-LEARNING_RATE) * Q_table[(oldObs[0][1], oldObs[0][2], oldObs[1][2])][action] + (LEARNING_RATE) * (reward + DISCOUNT_FACTOR * (max_a_prime(Q_table, obs)))

        episode_reward += reward # update episode reward

        index += 1
        # if we take more than 100 steps, end this iteration early (we are probably not making progress)
        if index > 100:
            done=True

    # decay the epsilon
    EPSILON *= EPSILON_DECAY

    # record the reward for this episode
    episode_reward_record.append(episode_reward) 

    if i%100 ==0 and i>0:
        print("Average reward for the last 100 iterations: " + str(sum(list(episode_reward_record))/100))
        print("epsilon: " + str(EPSILON) )



Average reward for the last 100 iterations: -89.26391252443506
epsilon: 0.9038873549665959
Average reward for the last 100 iterations: -88.73080416895395
epsilon: 0.8178301806491574
Average reward for the last 100 iterations: -84.04895409896336
epsilon: 0.7399663251239436
Average reward for the last 100 iterations: -75.66846459422796
epsilon: 0.6695157201007336
Average reward for the last 100 iterations: -73.40194100353149
epsilon: 0.6057725659163237
Average reward for the last 100 iterations: -69.1596364686203
epsilon: 0.548098260578011
Average reward for the last 100 iterations: -77.77323334448975
epsilon: 0.4959150020176678
Average reward for the last 100 iterations: -72.7534170313123
epsilon: 0.44869999946146477
Average reward for the last 100 iterations: -64.64710241432209
epsilon: 0.4059802359226587
Average reward for the last 100 iterations: -72.26314518707726
epsilon: 0.36732772934619257
Average reward for the last 100 iterations: -72.18309584603773
epsilon: 0.33235524492954527

In [9]:
to_test = B@B@B@B@C

def matrix_to_tuple(matrix):
    return (matrix[0][1], matrix[0][2], matrix[1][2])

Q_table[matrix_to_tuple(to_test)]

[-1.138397577870297,
 -1.1032005356227106,
 -1.1981417647058823,
 16.565255659473515]

In [10]:
# test with the other dataframe. 
test_df = pd.read_csv("heisenberg_data_test.csv")

In [11]:
for i in test_df:
    print(i)

val1
val2
val3
last_matrix


In [12]:
cur_matrix = np.array([[1, int(cur_row['val1']), int(cur_row['val2'])], [0, 1, int(cur_row['val3'])], [0, 0, 1]])

In [13]:
def matrix_to_num_steps(cur_matrix):
    index = 1
    for i in range(50):
        if (cur_matrix==A).all() or (cur_matrix==B).all() or (cur_matrix==C).all() or (cur_matrix==D).all():
            return i
        outputs = Q_table[matrix_to_tuple(cur_matrix)]
        # print(outputs)
        # if outputs==[0, 0, 0, 0]:
        #     # this is a problem because we haven't seen this state before 
        #     # in training so we have no idea how to handle it
        #     print("Problem.")
        index = np.argmax(outputs)
        if index==0:
            cur_matrix = cur_matrix @ A
        elif index==1:
            cur_matrix = cur_matrix @ B
        elif index==2:
            cur_matrix = cur_matrix @ C
        elif index==3:
            cur_matrix = cur_matrix @ D
    return 100

In [14]:
print(B@C@B)

matrix_to_num_steps(B@C@B)

[[ 1 -2 -1]
 [ 0  1  1]
 [ 0  0  1]]


2

In [15]:
mymat = B@B@C@B@D@A@A@C@A
print(mymat)


cur_matrix = D@D@B@D@A

matrix_to_num_steps(cur_matrix)

[[1 0 0]
 [0 1 1]
 [0 0 1]]


4

In [16]:
type(test_df['val1'][0])

numpy.int64

In [17]:
def test_Q_learning(row):
    cur_matrix = np.array([[1, int(row['val1']), int(row['val2'])], [0, 1, int(row['val3'])], [0, 0, 1]])
    return matrix_to_num_steps(cur_matrix)

test_df['num_moves_Q_learning_needs'] = test_df.apply(test_Q_learning, axis=1)

In [22]:
print("The proportion of starting positions in the test dataset that we can find a route to the origin that's <50 steps: ")
sum(test_df['num_moves_Q_learning_needs']!=100)/test_df.shape[0]

0.48914891489148915

In [25]:
print("Of these, the proportion of times where we learned a path that was < 20 moves: ")
# encouraging because all of these were generated as sequences of 30 moves
# so we've found significantly faster paths back to the origin for almost all moves that we find a path to the origin 
sum(test_df['num_moves_Q_learning_needs']<20)/sum(test_df['num_moves_Q_learning_needs']!=100)

Of these, the proportion of times where we learned a path that was < 20 moves: 


0.9989777141688816